## Video Demos

### Install av library

In [ ]:
!conda install av -c conda-forge -y

### Download and play video

In [ ]:
!curl https://raw.githubusercontent.com/MicrosoftLearning/AI-Introduction/master/files/Intro.mp4 -o video.mp4

In [ ]:
%%HTML
<video width="320" height="240" controls>
  <source src="video.mp4" type="video/mp4">
</video>

### Count frames and show the 25th one

In [ ]:
import av
container = av.open('video.mp4')

%matplotlib inline
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw

for frame in container.decode(video=0):
    if (frame.index == 25):
        img = frame.to_image()
        imshow(img)
frameCount = frame.index - 1
print(str(frameCount) + " frames")

## Face Tracking

### Create a Video Indexer API Subscription
https://videobreakdown.portal.azure-api.net/

In [ ]:
apiKey = "abcdefg1234567"

### Upload a Video for Processing

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

# We'll upload this video from GitHub to the Video Indexer
video_url='https://github.com/MicrosoftLearning/AI-Introduction/raw/master/files/vid.mp4'

headers = {
    # Request headers
    'Content-Type': 'multipart/form-data',
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'name': 'vid',
    'privacy': 'Private',
    'videoUrl': video_url,
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("POST", "/Breakdowns/Api/Partner/Breakdowns?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    # The response is a byte array - convert to a string and remove quotes
    vidId = data.decode("UTF-8").replace('"', '')
    print("Breakdown ID: " + vidId)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

### Check Status

In [ ]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s/State?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))


### View the Video

In [ ]:
from IPython.core.display import HTML

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s/PlayerWidgetUrl?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    vidUrl = data.decode("UTF-8").replace('"', '')
    print (vidUrl)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
HTML('<iframe width=600 height=400 src="%s"/>' % vidUrl )

### View the Video Breakdown

In [ ]:
import json

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    strData = data.decode("UTF-8")
    jData = json.loads(strData)
    print (json.dumps(jData, sort_keys=True, indent=2))
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))



### Get Details of Faces Identified in the Video

In [ ]:
print(json.dumps(jData["summarizedInsights"]["faces"], sort_keys=True, indent=2))
faceUrl = jData["summarizedInsights"]["faces"][0]["thumbnailFullUrl"]
print(faceUrl)
HTML('<img src="%s"/>' % faceUrl )

### View (and Edit) People Insights

In [ ]:
headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'widgetType': 'People',
    'allowEdit': 'True',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s/InsightsWidgetUrl?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    iUrl = data.decode("UTF-8").replace('"', '')
    print(iUrl)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

HTML('<iframe width=800 height=600 src="%s"/>' % iUrl )

### Reload Breakdown and Check Updated Face Details

In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json

headers = {
    # Request headers
    'Ocp-Apim-Subscription-Key': apiKey,
}

params = urllib.parse.urlencode({
    # Request parameters
    'language': 'en-US',
})

try:
    conn = http.client.HTTPSConnection('videobreakdown.azure-api.net')
    conn.request("GET", "/Breakdowns/Api/Partner/Breakdowns/%s?%s" % (vidId, params), "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    strData = data.decode("UTF-8")
    jData = json.loads(strData)
    print(json.dumps(jData["summarizedInsights"]["faces"], sort_keys=True, indent=2))
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))
